In [2]:
import numpy as np

def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)  # number of layers in the network
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.001
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters



In [3]:
def linear_forward(A_prev, W, b):
    Z = np.dot(W, A_prev) + b
    cache = (A_prev, W, b)
    return Z, cache

In [4]:
def sigmoid(Z):
    Z = np.clip(Z, -500, 500)  # Prevent overflow
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    A = np.maximum(0, Z)
    cache = Z
    return A, cache

In [5]:
def linear_activation_forward(A_prev, W, b, activation):
    Z, linear_cache = linear_forward(A_prev, W, b)
    if activation == "sigmoid":
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache


In [6]:
def L_model_forward(X,parameters):
    A = X
    caches = []
    for l in range(1, len(parameters) // 2):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation='relu')
        caches.append(cache)
    AL, cache = linear_activation_forward(A, parameters['W' + str(len(parameters) // 2)], parameters['b' + str(len(parameters) // 2)], activation='sigmoid')
    caches.append(cache)
    return AL, caches

In [7]:
def compute_cost(AL, Y):
    J = - (1/Y.shape[1]) * np.sum(Y * np.log(AL + 1e-15) + (1 - Y) * np.log(1 - AL + 1e-15))
    return np.squeeze(J)

In [8]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    return dA_prev, dW, db


In [9]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    return dZ

In [1]:
def linear_activation_backward(da, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(da, activation_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(da, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db


In [ ]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    epsilon = 1e-15
    dAL = - (np.divide(Y, AL + epsilon) - np.divide(1 - Y, 1 - AL + epsilon))
    current_cache = caches[L-1]
    dA_prev_L, dWL, dbL = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    grads[f"dW{L}"] = dWL
    grads[f"db{L}"] = dbL
    dA_current = dA_prev_L

    for l in reversed(range(L - 1)): # l va de L-2, L-3, ..., 0
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dA_current, current_cache, activation="relu")

        grads[f"dW{l + 1}"] = dW_temp
        grads[f"db{l + 1}"] = db_temp

        dA_current = dA_prev_temp

    return grads
